In [1]:
import pandas as pd
import numpy as np

In [3]:
click_columns = ['user_id','item_id','time']
train_click_T = pd.DataFrame()
test_click_T = pd.DataFrame()
test_qtime_T = pd.DataFrame()


# 第t个阶段，读取所有文件并合并
t = 4
for i in range(t):
    temp_train_click = pd.read_csv('../data/underexpose_train/underexpose_train_click-%s.csv'%(str(i)),sep=',',names = click_columns )
    temp_test_click = pd.read_csv('../data/underexpose_test/underexpose_test_click-%s.csv'%(str(i)),sep=',',names = click_columns)
    temp_test_qtime = pd.read_csv('../data/underexpose_test/underexpose_test_qtime-%s.csv'%(str(i)),sep=',',names = ['user_id','time'])
    train_click_T = train_click_T.append(temp_train_click)
    test_click_T = test_click_T.append(temp_test_click)
    test_qtime_T = test_qtime_T.append(temp_test_qtime)

# 拼接所有历史点击数据
data_click_T = pd.concat([train_click_T,test_click_T],axis = 0).reset_index(drop = True)


# 计算所有商品的流行度-历史点击次数排序
data_click_T_item = data_click_T.groupby('item_id',as_index= False)['user_id'].agg({'user_dis':'unique','click_cnt':'count'})
data_click_T_item_rank = data_click_T_item.sort_values(by = 'click_cnt',axis = 0,ascending = False)['item_id'].tolist()

In [18]:
data_click_T_item

,item_id,user_dis,click_cnt
0,1,"[10088, 1489, 19312, 13420]",4
1,2,"[31391, 19402, 10765, 18252]",4
2,3,"[5206, 24081, 18247, 8022, 2329, 5788, 10370, ...",27
3,4,"[29720, 34045, 8248, 6181]",4
4,8,"[4307, 4033, 18332, 2100]",4
...,...,...,...
71899,117502,"[9290, 9071, 3565, 3904, 4817, 13725, 21206, 1...",13
71900,117505,"[8492, 6611, 3501, 24379]",4
71901,117507,"[31534, 9119, 12705, 7545, 4489, 25210, 4951, ...",10
71902,117540,"[19445, 5276, 11680, 13207]",4


In [19]:
data_click_T_item_rank

[52766,
 111225,
 4340,
 21426,
 2420,
 114125,
 21279,
 12845,
 87254,
 7156,
 42651,
 77047,
 44534,
 1604,
 35452,
 6568,
 12061,
 114201,
 19326,
 20208,
 13845,
 34716,
 91838,
 76943,
 17287,
 52131,
 67536,
 4999,
 27885,
 14239,
 4230,
 14777,
 13302,
 80227,
 78364,
 114055,
 88605,
 49787,
 45198,
 29594,
 59903,
 104777,
 24025,
 110172,
 1518,
 51182,
 55349,
 88058,
 114108,
 4483,
 69717,
 5346,
 8180,
 103775,
 29931,
 42884,
 109233,
 102105,
 113569,
 10143,
 27337,
 8149,
 59996,
 23825,
 28665,
 77095,
 3654,
 11052,
 110872,
 111114,
 8076,
 6368,
 899,
 29813,
 71223,
 112489,
 75835,
 21399,
 35264,
 664,
 31054,
 733,
 41755,
 92379,
 19270,
 855,
 32712,
 62081,
 6731,
 15974,
 25836,
 69019,
 1609,
 49124,
 20333,
 61324,
 109125,
 26732,
 90006,
 36071,
 27787,
 36073,
 31776,
 69793,
 2248,
 20925,
 87107,
 37885,
 9338,
 14552,
 103605,
 107299,
 65393,
 79868,
 49939,
 67755,
 7709,
 88002,
 17417,
 85661,
 16391,
 20510,
 69749,
 6866,
 4207,
 57900,
 5629

In [20]:
def next_item_by(user_list):
    '''
    针对所有用户，计算历史上某个商品点击之后下一个被点击商品的list
    返回数据格式：{'item_id':{'next_item_1':'click_count1','next_item2':'click_count2'}}
    '''
    item_seq={}
    for user_id in user_list:
        temp = data_click_T[data_click_T.user_id == user_id]
        temp_1 = temp.sort_values(by = 'time',axis = 0,ascending = True)
        c = temp_1['item_id'].tolist()
        for index,item in enumerate(c):
            if index ==len(c)-1:break
            if item not in item_seq:
                item_seq[item] = {}
                if c[index+1] not in item_seq[item]:
                    item_seq[item][c[index+1]] = 1
                else:
                    item_seq[item][c[index+1]] += 1
            else:    
                if c[index+1] not in item_seq[item]:
                    item_seq[item][c[index+1]] = 1
                else:
                    item_seq[item][c[index+1]] += 1
    return item_seq

item_seq = next_item_by(data_click_T['user_id'].unique())

In [21]:
item_seq

{33354: {43218: 1,
  33354: 11,
  100466: 1,
  94723: 1,
  96512: 1,
  98319: 1,
  41951: 1,
  73113: 1,
  106807: 1,
  46881: 1,
  112708: 1,
  84108: 1},
 43218: {18: 1, 43218: 2, 21198: 1, 97990: 1, 76437: 1},
 18: {66446: 1,
  18: 9,
  36238: 1,
  108247: 1,
  21794: 1,
  20375: 1,
  53547: 1,
  93963: 1,
  77657: 1,
  42736: 1},
 66446: {3598: 1, 14341: 1, 92851: 1},
 3598: {55956: 1,
  17336: 1,
  23583: 1,
  90290: 1,
  116349: 10,
  101596: 1,
  87107: 8,
  116437: 11,
  12627: 6,
  3598: 16,
  94404: 1,
  82029: 3,
  1736: 3,
  53718: 4,
  110461: 1,
  45431: 2,
  38672: 1,
  3001: 3,
  82010: 1,
  3428: 2,
  34437: 1,
  7158: 1,
  115865: 1,
  94726: 1,
  28507: 2,
  108496: 1,
  113569: 2,
  1921: 1,
  37170: 1,
  64838: 1,
  62570: 1,
  115860: 1,
  94397: 1,
  11370: 1,
  113418: 1,
  11890: 1,
  107281: 1,
  88043: 1,
  2241: 1,
  59651: 1},
 55956: {93772: 1,
  55956: 33,
  4484: 1,
  58592: 3,
  77981: 1,
  11052: 1,
  114125: 1,
  45008: 1,
  31402: 1,
  72847: 1,
  38

In [22]:
def last_item(user_list):
    """
    获取测试集用户上一次点击的商品id
    返回数据格式：{'user_id':'last_click_item_id'}}
    """
    user_lastitem={}
    for user_id in user_list:
        temp =test_click_T[test_click_T.user_id == user_id]
        temp_1 = temp.sort_values(by = 'time',axis = 0,ascending = False).iloc[0:1,1:2].values[0][0]
        user_lastitem[user_id]= temp_1
    return user_lastitem

user_lastitem = last_item(test_click_T.user_id.tolist())


# 根据用户上一次点击的商品，把历史统计出来的点完这个商品还会点哪些的统计结果保存到对应user_id,并且把这个作为结果提交
user_nextitem_dict= {}
for user_id in user_lastitem.keys():
    item_id = user_lastitem[user_id]
    if item_id not in item_seq:
        user_nextitem_dict[user_id]= []
    else:
        next_item_dict = item_seq[item_id]
        temp =sorted(next_item_dict.items(),key = lambda x:x[1],reverse = True)
        next_item_list =[]
        for i in range(len(temp)):
            next_item_list.append(temp[i][0])
        user_nextitem_dict[user_id]= next_item_list



def deal(x):
    """对测试集的所有用户，用当前商品历史上下一次点击商品作为预测目标。如果长度不足50,则用所有商品流行度补全"""
    y=[]
    if len(user_nextitem_dict[x]) >=50:
        y = user_nextitem_dict[x][0:50]
    else:
        y=user_nextitem_dict[x]
        for i in data_click_T_item_rank:
            if i in y:continue
            y.append(i)
            if len(y)>=50:
                break
    return y
    
test_qtime_T['candidate'] = test_qtime_T['user_id'].apply(lambda x :deal(x))


"""生成对应的列"""
for i in range(50):
    if i < 9:
        col = 'item_id_0'+str(i+1)
    else:
        col = 'item_id_'+str(i+1)
    test_qtime_T[col] = test_qtime_T['candidate'].apply(lambda x: x[i])    

test_qtime_T = test_qtime_T.drop(columns=['time','candidate'])

In [23]:
test_qtime_T

,user_id,item_id_01,item_id_02,item_id_03,item_id_04,item_id_05,item_id_06,item_id_07,item_id_08,item_id_09,...,item_id_41,item_id_42,item_id_43,item_id_44,item_id_45,item_id_46,item_id_47,item_id_48,item_id_49,item_id_50
0,11,8052,26711,31797,51361,103317,81088,82016,85475,69791,...,73755,41143,78804,86802,23323,2949,99219,52766,111225,4340
1,22,101862,98489,101900,32368,15935,108501,90388,87484,108888,...,1604,35452,6568,12061,114201,19326,20208,13845,34716,91838
2,44,25164,26509,25454,2549,110172,42253,79386,25356,52766,...,13302,80227,78364,114055,88605,49787,45198,29594,59903,104777
3,55,51356,110324,33921,87144,87011,14626,90614,61953,7259,...,20208,13845,34716,91838,76943,17287,52131,67536,4999,27885
4,66,43836,85661,42985,89864,63510,82418,90006,73444,36157,...,110448,2942,15690,52997,36648,78868,66752,10676,104128,64884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,35313,8642,68803,2028,43436,52766,111225,4340,21426,2420,...,88605,49787,45198,29594,59903,104777,24025,110172,1518,51182
1671,35335,32268,110925,106244,51902,16454,16233,52766,111225,4340,...,78364,114055,88605,49787,45198,29594,59903,104777,24025,110172
1672,35357,56635,11347,75033,87121,51973,52766,111225,4340,21426,...,114055,88605,49787,45198,29594,59903,104777,24025,110172,1518
1673,35379,52410,68689,41161,52766,111225,4340,21426,2420,114125,...,49787,45198,29594,59903,104777,24025,110172,1518,51182,55349
